In [2]:
import pandas as pd
import numpy as np
import csv
import string

## Get relations with their relation type for training

In [3]:
#training data - relations 
relations = pd.read_csv("relations.csv") 

#print(relations)
relations.head()

,Type,Id_1,Id_2,Reverse
0,USAGE,H01-1001.5,H01-1001.7,REVERSE
1,USAGE,H01-1001.9,H01-1001.10,NaN
2,PART_WHOLE,H01-1001.14,H01-1001.15,REVERSE
3,MODEL-FEATURE,H01-1017.4,H01-1017.5,NaN
4,PART_WHOLE,H01-1041.3,H01-1041.4,REVERSE


In [15]:
len(relations)

1228

- There are 1228 relation pairs in the training set
- context.pkl contains the entity context represented by GloVe word embeddings
- context_txt.pkl is a dataframe which consists of the absID (the id of the abstract the entity is extracted from), entitiyID (the Id of the entity), context (entity with the sorrounding context)  

In [4]:
import pickle
context = pd.read_pickle('context.pkl')
context.head()

,entity_Id,context_vector
0,H01-1001.1,"[[-0.40088, 0.38661, -0.53597, -0.081736, 0.37..."
1,H01-1001.2,"[[-0.10137, -0.27174, -0.31147, -0.75354, 0.05..."
2,H01-1001.3,"[[-0.52579, 0.3754, -0.41452, -0.48975, 0.0334..."
3,H01-1001.4,"[[-0.44399, 0.12817, -0.25247, -0.18582, -0.16..."
4,H01-1001.5,"[[-1.0001, 0.45166, 0.38212, -0.098643, 0.1492..."


In [5]:
context_txt = pd.read_pickle("context_txt.pkl")
context_txt.head()

,absID,entityID,context
0,H01-1001,H01-1001.1,oral communication is ubiquitous
1,H01-1001,H01-1001.2,development of storage media and networks one ...
2,H01-1001,H01-1001.3,store a conversation for documentation.
3,H01-1001,H01-1001.4,in a large database traditional information
4,H01-1001,H01-1001.5,database traditional information retrieval tec...


In [6]:
relations_vector_1 = []
for i in range(len(relations)):
    for j in range(len(context)):
        if relations['Id_1'][i] == context['entity_Id'][j]:
            relations_vector_1.append(context['context_vector'][j])

In [13]:
#print(relations_vector_1[0])
len(relations_vector_1[0])

7

In [7]:
relations_vector_2 = []
for k in range(len(relations)):
    for l in range(len(context)):
        if relations['Id_2'][k] == context['entity_Id'][l]:
            relations_vector_2.append(context['context_vector'][l])

In [8]:
#print(relations_vector_2[0])
len(relations_vector_2[0])

5

In [9]:
data = pd.DataFrame({'entity_id_1': relations['Id_1'], 'entity_context_1': relations_vector_1,
                     'entity_id_2': relations['Id_2'], 'entity_context_2': relations_vector_2,
                     'label': relations['Type']}) 

This is the entity pair from the relations file. 
Except entity pairs are also attached in case we want to train the model with the GloVe text representations. 

In [34]:
data.head()

,entity_id_1,entity_context_1,entity_id_2,entity_context_2,label
0,H01-1001.5,"[[-1.0001, 0.45166, 0.38212, -0.098643, 0.1492...",H01-1001.7,"[[0.019734, -0.072078, 0.66418, 0.050783, 0.01...",USAGE
1,H01-1001.9,"[[-0.3836, -0.14726, 0.18546, -0.39794, 0.3098...",H01-1001.10,"[[-0.19396, -0.2022, 0.12097, 0.36407, 0.16746...",USAGE
2,H01-1001.14,"[[-0.29712, 0.094049, -0.096662, -0.344, -0.18...",H01-1001.15,"[[-1.0001, 0.45166, 0.38212, -0.098643, 0.1492...",PART_WHOLE
3,H01-1017.4,"[[-0.076947, -0.021211, 0.21271, -0.72232, -0....",H01-1017.5,"[[-0.015955, -0.63705, -0.29999, -0.40689, -0....",MODEL-FEATURE
4,H01-1041.3,"[[-0.20145, 0.40871, -0.096877, -0.55708, -0.1...",H01-1041.4,"[[-0.076947, -0.021211, 0.21271, -0.72232, -0....",PART_WHOLE


In [10]:
data_to_encode = data.drop(columns =['entity_id_1'])
data_to_encode = data_to_encode.drop(columns =['entity_id_2'])

In [39]:
data_to_encode.head()

,entity_context_1,entity_context_2,label
0,"[[-1.0001, 0.45166, 0.38212, -0.098643, 0.1492...","[[0.019734, -0.072078, 0.66418, 0.050783, 0.01...",USAGE
1,"[[-0.3836, -0.14726, 0.18546, -0.39794, 0.3098...","[[-0.19396, -0.2022, 0.12097, 0.36407, 0.16746...",USAGE
2,"[[-0.29712, 0.094049, -0.096662, -0.344, -0.18...","[[-1.0001, 0.45166, 0.38212, -0.098643, 0.1492...",PART_WHOLE
3,"[[-0.076947, -0.021211, 0.21271, -0.72232, -0....","[[-0.015955, -0.63705, -0.29999, -0.40689, -0....",MODEL-FEATURE
4,"[[-0.20145, 0.40871, -0.096877, -0.55708, -0.1...","[[-0.076947, -0.021211, 0.21271, -0.72232, -0....",PART_WHOLE


In [11]:
relations_txt_1 = []
for i in range(len(relations)):
    for j in range(len(context_txt)):
        if relations['Id_1'][i] == context_txt['entityID'][j]:
            relations_txt_1.append(context_txt['context'][j])

In [20]:
print(relations_txt_1[0])
len(relations_txt_1)

database traditional information retrieval techniques use a


1228

In [12]:
relations_txt_2 = []
for k in range(len(relations)):
    for l in range(len(context_txt)):
        if relations['Id_2'][k] == context_txt['entityID'][l]:
            relations_txt_2.append(context_txt['context'][l])

In [13]:
print(relations_txt_2[0])
len(relations_txt_2)

histogram of keywords as the


1228

In [14]:
text_data = pd.DataFrame({'entity_id_1': relations['Id_1'], 'entity_text_1': relations_txt_1,
                     'entity_id_2': relations['Id_2'], 'entity_text_2': relations_txt_2,
                     'label': relations['Type']}) 

In [31]:
text_data.head()

,entity_id_1,entity_text_1,entity_id_2,entity_text_2,label
0,H01-1001.5,database traditional information retrieval tec...,H01-1001.7,histogram of keywords as the,USAGE
1,H01-1001.9,representation but oral communication may offer,H01-1001.10,offer additional indices such as,USAGE
2,H01-1001.14,a large database of tv,H01-1001.15,database of tv shows emotions and,PART_WHOLE
3,H01-1017.4,of a distributed message-passing infrastructur...,H01-1017.5,infrastructure for dialogue systems which all,MODEL-FEATURE
4,H01-1041.3,laboratory) the cclinc korean-to-english trans...,H01-1041.4,of two core modules language understanding,PART_WHOLE


In [15]:
text_data_to_encode = text_data.drop(columns =['entity_id_1'])
text_data_to_encode = text_data_to_encode.drop(columns =['entity_id_2'])

In [16]:
#store this data for later use in the bert model
text_data_to_encode.to_pickle("entity_pair.pkl")

In [25]:
text_data_to_encode.head()

,entity_text_1,entity_text_2,label
0,database traditional information retrieval tec...,histogram of keywords as the,USAGE
1,representation but oral communication may offer,offer additional indices such as,USAGE
2,a large database of tv,database of tv shows emotions and,PART_WHOLE
3,of a distributed message-passing infrastructur...,infrastructure for dialogue systems which all,MODEL-FEATURE
4,laboratory) the cclinc korean-to-english trans...,of two core modules language understanding,PART_WHOLE


In [36]:
type(text_data_to_encode['entity_text_1'][1])

0       database traditional information retrieval tec...
1         representation but oral communication may offer
2                                  a large database of tv
3       of a distributed message-passing infrastructur...
4       laboratory) the cclinc korean-to-english trans...
                              ...                        
1223                       whether its sentences are used
1224                             bound of accuracy of 77%
1225          proposed, allowing non-terminals to consist
1226                strongly adequate grammar for crossed
1227                  an existing parsing method for gpsg
Name: entity_text_1, Length: 1228, dtype: object


Merge the two entity columns into one. 

In [41]:
entity_pair = text_data_to_encode['entity_text_1'] + ' ' + text_data_to_encode['entity_text_2']
data_to_train = pd.DataFrame({'entity_pair': entity_pair, 'label':text_data_to_encode['label'] })
data_to_train.head()

,entity_pair,label
0,database traditional information retrieval tec...,USAGE
1,representation but oral communication may offe...,USAGE
2,a large database of tv database of tv shows em...,PART_WHOLE
3,of a distributed message-passing infrastructur...,MODEL-FEATURE
4,laboratory) the cclinc korean-to-english trans...,PART_WHOLE


In [42]:
data_to_train.to_pickle("train.pkl")